In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Data Preprocessing

In [0]:
import pandas as pd
df=pd.read_csv("drive/My Drive/Colab Notebooks/Final Project/Task 1/train.csv")
df.head()

In [ ]:
traindf = df.sample(frac=0.75, random_state=777)   # 隨機將train.csv裡的檔案分成train:validation=3:1
validf = df.drop(traindf.index)

display(traindf.head())
display(validf.head())
print('各DataFrame 大小:', len(df), len(traindf), len(validf))

In [3]:
testdf=pd.read_csv("drive/My Drive/Colab Notebooks/Final Project/Task 1/testName.csv")
testdf.head()

,Pic Name
0,test_Depth_9280.png
1,20191024_10852_Depth_112034.png
2,test_Depth_14496.png
3,20190531_105600_Depth_5472.png
4,20190531_100725_Depth_20817.png


In [11]:
from keras.preprocessing.image import ImageDataGenerator

# base_dir = '.\\train_img'
# train_dir = os.path.join(base_dir, 'train_img')
train_dir = 'drive/My Drive/Colab Notebooks/Final Project/Task 1/totalDepthPic'
valid_dir = 'drive/My Drive/Colab Notebooks/Final Project/Task 1/totalDepthPic'
test_dir = 'drive/My Drive/Colab Notebooks/Final Project/Task 1/totalDepthPic'

train_datagen = ImageDataGenerator(
        rotation_range=15,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255., 
        # validation_split=0.25
)
train_generator = train_datagen.flow_from_dataframe( 
    dataframe=traindf, 
    directory=train_dir, 
    x_col='Pic Name', 
    y_col='Total rebars', 
    has_ext=False, 
    # subset="training", 
    class_mode="raw", 
    batch_size=20,
    target_size=(150, 150)
    ) 
vaild_datagen = ImageDataGenerator(
        rescale=1./255.
)
validation_generator = vaild_datagen.flow_from_dataframe( 
    dataframe=validf, 
    directory=valid_dir,
    x_col='Pic Name', 
    y_col='Total rebars', 
    has_ext=False, 
    # subset="validation", 
    class_mode="raw",
    batch_size=20,
    target_size=(150, 150)
    )

test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory=test_dir,
    x_col='Pic Name',
    target_size=(150, 150),
    color_mode='rgb',
    shuffle=False,
    class_mode=None
)

Found 517 validated image filenames.
Found 172 validated image filenames.
Found 101 validated image filenames.


In [7]:
import keras.applications.resnet_v2
from keras.applications.resnet50 import ResNet50

basemodel = ResNet50(weights='imagenet',
                     include_top=False,
                     input_shape=(150, 150, 3))














102858752/102853048 [==============================] - 9s 0us/step


In [ ]:
import numpy as np

def extract_features(generator, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    batch_size = generator.batch_size
    # generator = datagen.flow_from_directory(
    #     directory,
    #     target_size=(150, 150),
    #     batch_size=batch_size,
    #     class_mode='binary')
    i = 0

    if generator.class_mode is None:
      for inputs_batch in generator:
        features_batch = basemodel.predict(inputs_batch)
        features[i : (i + features_batch.shape[0])] = features_batch   # 避免最後一個batch的shape不合
        i += features_batch.shape[0]
        if i >= sample_count:
          # Note that since generators yield data indefinitely in a loop,
          # we must `break` after every image has been seen once.
          break
      return features
    else:
      for inputs_batch, labels_batch in generator:
        features_batch = basemodel.predict(inputs_batch)
        # features[i * batch_size : (i + 1) * batch_size] = features_batch
        features[i : (i + features_batch.shape[0])] = features_batch   # 避免最後一個batch的shape不合
        # labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        labels[i : (i + features_batch.shape[0])] = labels_batch
        i += features_batch.shape[0]
        if i >= sample_count:
          break
    print()
    return features, labels

train_features, train_labels = extract_features(train_generator, train_generator.samples)
validation_features, validation_labels = extract_features(validation_generator, validation_generator.samples)
test_features = extract_features(test_generator, test_generator.samples)

In [ ]:
train_features = np.reshape(train_features, (train_generator.samples, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (validation_generator.samples, 4 * 4 * 512))
test_features = np.reshape(test_features, (test_generator.samples, 4 * 4 * 512))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.3))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation=None))

In [9]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='mse',
              metrics=['mse'])

In [12]:
model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/50


ValueError: ignored

In [0]:
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]